In [6]:
import requests
import io
import json
import csv
from requests import Response
from requests.exceptions import HTTPError,ConnectionError,Timeout
from pydantic import BaseModel,RootModel,Field,field_validator,field_serializer
from datetime import datetime
from pprint import pprint

youbike_url='https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
try:
    respons:Response=requests.request('GET',youbike_url)
    respons.raise_for_status()
except ConnectionError:
    print("連線伺服器發生錯誤")
except Timeout:
    print("伺服器太忙,沒有回應")
except HTTPError:
    print("伺服器回應連線錯誤")
except Exception:
    print("不知名的錯誤")
else:
    content=respons.text




bike=[item for item in content if "so4cj6fm " in item["觀光遊憩區"]]
key_bike:list[str]=list(bike[0].keys())

with open("玉山國家公園.csv","w",encoding="utf-8",newline="") as file:
    dictWriter=csv.DictWriter(file,key_bike)
    dictWriter.writeheader()
    dictWriter.writerows(bike)




class Station(BaseModel):
    站名:str=Field(alias='sna')
    行政區:str=Field(alias='sarea')
    日期:datetime=Field(alias='mday')
    地址:str=Field(alias='ar')
    狀態:str=Field(alias='act')
    總車輛數:int=Field(alias='total')
    可借:int=Field(alias='available_rent_bikes')
    可還:int=Field(alias='available_return_bikes')

    @field_validator('站名')
    @classmethod
    def slice_str(cls,name:str):
        return name[11:]
    
    @field_validator('狀態',mode="before")
    @classmethod
    def slice_str(cls,status:str):
        if status=="1":
            return "營業中"
        else:
            return "維修中"
        
    @field_serializer("日期")
    def date_serializer(self,date:datetime)->str:
        return date.strftime(f"中華民國{date.year-1911}年%m月%d日 %H時%M分%S秒")
class Youbikes(RootModel):
    root:list[Station]

youbikes:Youbikes=Youbikes.model_validate_json(content)
youbikes_list=youbikes.model_dump()
pprint(youbikes_list)




content

TypeError: string indices must be integers, not 'str'